In [65]:
import pandas as pd
import re
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [66]:
df = pd.read_csv('/home/hackathon_2023/data/enron/emails.csv')

In [67]:
pattern1 = r"Message-ID: (?P<message_id>[\w\W]*)\nDate: (?P<date>[\w\W]*)\nFrom: (?P<from>[\w\W]*)\nTo: (?P<to>[\w\W]*)\nSubject: (?P<subject>[\w\W]*)\nMime-Version: (?P<mime_version>[\w\W]*)\nContent-Type: (?P<content_type>[\w\W]*)" \
          r"\nContent-Transfer-Encoding: (?P<content_encoding>[\w\W]*)\nX-From: (?P<x_from>[\w\W]*)\nX-To: (?P<x_to>[\w\W]*)\nX-cc: (?P<x_cc>[\w\W]*)\nX-bcc: (?P<x_bcc>[\w\W]*)\nX-Folder: (?P<x_folder>[\w\W]*)\nX-Origin: (?P<x_origin>[\w\W]*)" \
          r"\nX-FileName: (?P<x_filename>.*)\n(?P<body>[\w\W]*)"

empty_cell = {'message_id': None,
 'date': None,
 'from': None,
 'to': None,
 'subject': None,
 'mime_version': None,
 'content_type': None,
 'content_encoding': None,
 'x_from': None,
 'x_to': None,
 'x_cc': None,
 'x_bcc': None,
 'x_folder': None,
 'x_origin': None,
 'x_filename': None,
 'body': None}

def parse_re1(txt):
    
    m = re.match(pattern1, txt)
    if m is None:
        return None
    return m.groupdict()

In [68]:
parsed = df['message'].parallel_apply(parse_re1)

In [69]:
pattern2 = r"Message-ID: (?P<message_id>[\w\W]*)\nDate: (?P<date>[\w\W]*)\nFrom: (?P<from>[\w\W]*)\nSubject: (?P<subject>[\w\W]*)\nMime-Version: (?P<mime_version>[\w\W]*)\nContent-Type: (?P<content_type>[\w\W]*)" \
          r"\nContent-Transfer-Encoding: (?P<content_encoding>[\w\W]*)\nX-From: (?P<x_from>[\w\W]*)\nX-To: (?P<x_to>[\w\W]*)\nX-cc: (?P<x_cc>[\w\W]*)\nX-bcc: (?P<x_bcc>[\w\W]*)\nX-Folder: (?P<x_folder>[\w\W]*)\nX-Origin: (?P<x_origin>[\w\W]*)" \
          r"\nX-FileName: (?P<x_filename>.*)\n(?P<body>[\w\W]*)"

def parse_re2(txt):
    
    m = re.match(pattern2, txt)
    if m is None:
        return None
    return m.groupdict()

In [70]:
df_else = df.loc[parsed.isna()]['message'].apply(parse_re2)

In [71]:
df_else.isna().sum()

0

In [72]:
parsed[parsed.isna()] = df_else

In [73]:
df_parsed = pd.DataFrame(parsed.tolist())

In [74]:
df_parsed

,message_id,date,from,to,subject,mime_version,content_type,content_encoding,x_from,x_to,x_cc,x_bcc,x_folder,x_origin,x_filename,body
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,\nHere is our forecast\n\n
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,\nTraveling to have a business meeting takes t...
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,\ntest successful. way to go!!!
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"\nRandy,\n\n Can you send me a schedule of the..."
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,\nLet's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517396,<26807948.1075842029936.JavaMail.evans@thyme>,"Wed, 28 Nov 2001 13:30:11 -0800 (PST)",john.zufferli@enron.com,kori.loibl@enron.com,Trade with John Lavorato,1.0,text/plain; charset=us-ascii,7bit,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Loibl, Kori </O=ENRON/OU=NA/CN=RECIPIENTS/CN=K...",,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,john zufferli 6-26-02.PST,\nThis is a trade with OIL-SPEC-HEDGE-NG (John...
517397,<25835861.1075842029959.JavaMail.evans@thyme>,"Wed, 28 Nov 2001 12:47:48 -0800 (PST)",john.zufferli@enron.com,john.lavorato@enron.com,Gas Hedges,1.0,text/plain; charset=us-ascii,7bit,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,john zufferli 6-26-02.PST,\nSome of my position is with the Alberta Term...
517398,<28979867.1075842029988.JavaMail.evans@thyme>,"Wed, 28 Nov 2001 07:20:00 -0800 (PST)",john.zufferli@enron.com,dawn.doucet@enron.com,RE: CONFIDENTIAL,1.0,text/plain; charset=us-ascii,7bit,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Doucet, Dawn </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,john zufferli 6-26-02.PST,\n2\n\n -----Original Message-----\nFrom: \tDo...
517399,<22052556.1075842030013.JavaMail.evans@thyme>,"Tue, 27 Nov 2001 11:52:45 -0800 (PST)",john.zufferli@enron.com,jeanie.slone@enron.com,Calgary Analyst/Associate,1.0,text/plain; charset=us-ascii,7bit,"Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",,,"\ExMerge - Zufferli, John\Sent Items",ZUFFERLI-J,john zufferli 6-26-02.PST,\nAnalyst\t\t\t\t\tRank\n\nStephane Brodeur\t\...


In [75]:
df_parsed.to_parquet('/home/hackathon_2023/data/enron/emails.parquet')

In [76]:
df_parsed['body']

0                               \nHere is our forecast\n\n 
1         \nTraveling to have a business meeting takes t...
2                          \ntest successful.  way to go!!!
3         \nRandy,\n\n Can you send me a schedule of the...
4                     \nLet's shoot for Tuesday at 11:45.  
                                ...                        
517396    \nThis is a trade with OIL-SPEC-HEDGE-NG (John...
517397    \nSome of my position is with the Alberta Term...
517398    \n2\n\n -----Original Message-----\nFrom: \tDo...
517399    \nAnalyst\t\t\t\t\tRank\n\nStephane Brodeur\t\...
517400    \ni think the YMCA has a class that is for peo...
Name: body, Length: 517401, dtype: object